## Imports

In [ ]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import pymssql
import time
import numpy as np
import statistics

load_dotenv()

## Database Connection Helper Functions

In [ ]:
# Create database connection
def connect_to_db():
    conn = pymssql.connect(host=os.environ['db_host'], database=os.environ['db_name'])
    cursor = conn.cursor()
    return conn, cursor

# Close database connection
def close_db_connection(conn, cursor):
    cursor.close()
    conn.close()

## Request Ticker Data From Polygon API

In [ ]:
# Request data from api
def call_api(ticker, info, start, end):
    
    # Creating Request
    key = os.environ['polygon_api']
    url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start}/{end}?adjusted=true&sort=asc&apiKey={key}'

    # Retrieving data
    response = requests.get(url=url)
    data = pd.json_normalize(response.json()['results'])

    # Forrmat date column
    data.t = data.t.apply(lambda x: datetime.fromtimestamp(x/1e3).date().strftime('%Y-%m-%d'))

    # Map useful column names
    column_map = {
    'c': 'Close', 
    'h': 'High', 
    'l': 'Low', 
    'n': 'Transactions', 
    'o': 'Open', 
    'otc': 'OTC', 
    't': 'Date', 
    'v': 'Volume', 
    'vw': 'Price' # Volume Weighted Average Price
    }
    data.columns = data.columns.map(column_map)

    # Add missing dates between 2023-01-01 and Today
    date_range = pd.date_range(start='2023-01-01', end=datetime.today().strftime('%Y-%m-%d'))
    data.Date = data.Date.astype('datetime64[D]')

    # Filter to only include data after 2023-01-01
    data = data[data.Date >= '2023-01-01']

    # Fill empty dates with most recent data
    data = data.set_index('Date').reindex(date_range).resample('D').ffill().bfill().reset_index().rename(columns={'index':'Date'}).ffill()
    
    # Format date to YYYY-MM-DD
    data.Date = data.Date.apply(lambda x: x.strftime('%Y-%m-%d'))

    # Indices have no volume weighted price or volume
    # Calculate average price
    if 'Price' not in data.columns:
        data['Price'] = (data.Open + data.High + data.Low + data.Close) / 4
        data['Volume'] = None
    
    # Add ticker-specific data to dataframe
    data['Investment Type'] = info['type']
    data['Ticker'] = ticker
    data['Company'] = info['name']
    data = data[['Investment Type', 'Ticker','Company', 'Date', 'Price', 'Volume']]
    data['amount'] = info['amount']

    # Return resulting dataframe
    return data

## Further Preprocess with Calculated Columns

In [ ]:
# Add calculated columns to data
def calculate_columns(data):

    # 10 and 100 day moving averages
    data['ma10'] = data.Price.rolling(window=10).mean()
    data['ma100'] = data.Price.rolling(window=100).mean()

    # Total and Percentage change (day over day)
    data['change'] = data.Price.diff(periods=1)
    data['pct_change'] = data.Price.pct_change()

    # Fill NaN with None for SQL Server compatibility
    data = data.replace({np.nan: None})
    
    # Return updated dataframe
    return data

#### Beta Coefficient calculation function

In [ ]:
# Beta Coefficient
def beta(data, market):
    # Select common dates for comparison
    common = pd.merge(data[['Price', 'Date']], market[['Price', 'Date']], on='Date', how='inner')

    # Convert to percent change
    target = common.Price_x.pct_change().dropna()
    base = common.Price_y.pct_change().dropna()
    
    # Return Beta Coefficient
    return statistics.covariance(target, base) / statistics.variance(base)


## Final Preprocessing, split data into tables, insert into OLAP server

In [ ]:
# Insert data into database
def insert_data(conn, cursor, data, market):

	# Ticker metrics for investment_agg_facts table
	first_price = data.iloc[0].Price
	last_price = data.iloc[len(data)-1].Price
	first_date = data.iloc[0].Date
	last_date = data.iloc[len(data)-1].Date
	beta_val = beta(data, market)

	# Ticker info for investment_dimension table
	investments = data[['Ticker','Company', 'Investment Type']].drop_duplicates().apply(lambda x: x.str.title())
	investments['Ticker'] = investments['Ticker'].str.upper()
	investments['amount'] = data['amount']
	investments['shares'] = data['amount'] / first_price
	investments = [tuple(i) for i in investments.values]

	# Update investment_dimension table
	cursor.executemany("""
MERGE investment_dimension AS t
USING (
	VALUES 
		(%s, %s, %s, %d, %d)
	) AS s (ticker, company, investment_type, amount, shares)
ON t.ticker = s.ticker
WHEN NOT MATCHED BY target THEN
	INSERT VALUES (ticker, company, investment_type, amount, shares)
WHEN MATCHED THEN
	UPDATE SET 
		t.company = s.company, 
		t.investment_type = s.investment_type,
        t.amount = s.amount,
		t.shares = s.shares;
""", investments)
	# Save data in SQL Server
	conn.commit()

	# Create map of ticker name to id
	company_map = {}
	cursor.execute("Select * FROM investment_dimension;")
	for id, ticker, company, investment_type, amount, shares in cursor.fetchall():
		company_map[ticker] = id

	# Map foreign key ticker id
	data['investment_id'] = data['Ticker'].map(company_map)

	# Select necessary columns for price_facts table
	prices = data[['Price', 'Volume', 'investment_id', 'Date', 'ma10', 'ma100', 'change', 'pct_change']].copy()

	# Calculate value of portfolio investment from number of shares and price
	prices['value'] = investments[0][4] * prices['Price']

	# Calculate return of investment from value and original cost
	prices['return'] = prices['value'] - investments[0][3]

	# Calculate percent change weighted by portfolio fraction
	prices['weighted_pct'] = prices['pct_change'] * data['amount'] / 100000

	# Fill NaN with None for SQL Server Compatibility
	prices = prices.replace({np.nan: None})
	prices = [tuple(i) for i in prices.values]

    # Update price_facts table
	cursor.executemany("""
MERGE price_facts AS t
USING (
	VALUES
		(%d, %d, %d, %s, %d, %d, %d, %d, %d, %d, %d)
	) AS s (price, volume, investment_id, date, ma10, ma100, [change], pct_change, value, [return], weighted_pct)
ON (t.date = s.date AND t.investment_id = s.investment_id)
WHEN NOT MATCHED BY target THEN
	INSERT VALUES (price, volume, investment_id, date, ma10, ma100, [change], pct_change, value, [return], weighted_pct);
""", prices)
    
	# Update investment_agg_facts table
	cursor.execute(f"""
MERGE investment_agg_facts AS t
USING (
    VALUES
        ({company_map[data.Ticker.unique()[0]]}, {first_price}, {last_price}, '{first_date}', '{last_date}', {beta_val})    
    ) AS s (invest_id, first_price, last_price, first_date, last_date, beta)
ON t.invest_id = s.invest_id
WHEN NOT MATCHED BY target THEN
	INSERT VALUES (invest_id, first_price, last_price, first_date, last_date, beta)
WHEN MATCHED THEN
	UPDATE SET
		t.last_price = s.last_price, 
        t.last_date = s.last_date, 
		t.beta = s.beta;
""")
	# Save data in SQL Server
	conn.commit()

## Extract, Transform and Load into OLAP Server

In [ ]:
# Define tickers and timeframe
tickers = {'NVDA': {'type':'stock', 'name': 'Nvidia Corp', 'amount': 20000},
           'CRWD': {'type':'stock', 'name': 'Crowdstrike Holdings Inc', 'amount': 15000}, 
           'MU': {'type':'stock', 'name': 'Micron Technology Inc', 'amount': 15000}, 
           'SATO': {'type':'etf', 'name': 'Invesco Alerian Galaxy Crypto Economy ETF', 'amount': 20000},
           'SPY': {'type':'etf', 'name': 'SPDR S&P 500 ETF Trust', 'amount': 10000}, 
           'C:USDTRY': {'type':'forex', 'name': 'USD/TRY', 'amount': 10000}, 
           'X:SOLUSD': {'type':'crypto', 'name': 'SOL/USD', 'amount': 10000}, 
           'I:NDX': {'type':'index', 'name': 'Nasdaq-100', 'amount': 0}}
start = '2023-01-01'
end = datetime.today().strftime('%Y-%m-%d')

# Get broad market index data (NASDAQ 100)
market_index = call_api('I:NDX', {'type':'index', 'name': 'Nasdaq-100', 'amount': 0}, start, end)

# Connect to SQL Server
conn, cursor = connect_to_db()

# For each ticker
for ind, (ticker, info) in enumerate(tickers.items()):
    # Pause to stay within api rate limits
    if (ind + 1) % 5 == 0:
        time.sleep(60)

    # Query API and Preprocess
    data = call_api(ticker, info, start, end)

    # Add calculated columns to data
    data = calculate_columns(data)

    # Insert into DB
    insert_data(conn, cursor, data, market_index)

# Close DB Connection
close_db_connection(conn, cursor)